In [41]:
# 先知晓一下当前的目录地址
!pwd

/data/home/innovation/ouyangmalin/GW data analysis /baseline/BBH/test 1


In [42]:
# https://git.ligo.org/lscsoft/lalsuite/-/issues/300
import lal
lal.__version__

'7.6.1'

In [43]:
import torch
torch.__version__

'2.5.1+cu124'

In [44]:
from main import *
from utils import *

In [45]:
from matplotlib import rcParams, rc
from matplotlib import gridspec
params = {'backend': 'pdf',
          'axes.labelsize': 12, #坐标轴标签字体
          'font.size': 14,
          'axes.linewidth':1,
          'legend.fontsize': 13, #图例字体大小
          'xtick.labelsize': 12,
          'ytick.labelsize': 12, #刻度标签字体
          'ytick.major.pad': 4,
          'xtick.major.pad': 4,
          'xtick.major.width': 1,
          'ytick.major.width': 1,
          'xtick.minor.width': 1,
          'ytick.minor.width': 1,
          'xtick.major.size': 8,
          'ytick.major.size': 8,
          'xtick.minor.size': 5,
          'ytick.minor.size': 5,
          'text.usetex': False,
          'font.family':'serif', #字体
          # free font similar to Helvetica
		  # 'font.sans-serif':'Helvetica',
		  # 'text.latex.preamble':[r'\usepackage{amsmath}',r'\usepackage{amssymb}']
		  }
rcParams.update(params)

In [46]:
#parameters setting
fs = 8192 
T_obs = 1
snr=20

In [47]:
#generate PSD
dets=['H1']
psds = [gen_psd(fs,T_obs,op='AdvDesign',det=d) for d in dets]

#generate noise
psd =gen_psd(fs,T_obs)
ts_noise=np.array([gen_noise(fs,T_obs,psd.data.data) for psd in psds]).reshape(1,-1)
ts_noise_whiten=np.array([whiten_data(t,T_obs,fs,psd.data.data) for t,psd in zip(ts_noise,psds)]).reshape(1,-1)#white

#generate singal
par_new= gen_par(fs,T_obs)
ts_new,_,_ = gen_bbh(fs,T_obs,psds,par=par_new,snr=snr)
ts_new_whiten=np.array([whiten_data(t,T_obs,fs,psd.data.data) for t,psd in zip(ts_new,psds)]).reshape(1,-1)

Sat Dec 28 19:38:53 2024: using astrophysical logarithmic mass distribution
Sat Dec 28 19:38:53 2024: selected bbh masses = 30.32045914816183,9.19103575560883 (chirp mass = 14.051265673903949)
Sat Dec 28 19:38:53 2024: selected bbh cos(inclination) = -0.9800794563248911
Sat Dec 28 19:38:53 2024: selected bbh polarisation = 0.8122358978935057
Sat Dec 28 19:38:53 2024: selected bbh reference phase = 1.7117878897975851
Sat Dec 28 19:38:53 2024: selected bbh sky position = 2.993397377686586,0.6998229436827307
Sat Dec 28 19:38:53 2024: selected bbh peak amplitude time = 0.69140625
Sat Dec 28 19:38:53 2024: signal enters segment at 37.55726527447442 Hz
Sat Dec 28 19:38:53 2024: computed starting frequency = 37.55726527447442 Hz


Sat Dec 28 19:38:53 2024: computed H1 Earth centre time delay = 0.0012832599832278344
Sat Dec 28 19:38:53 2024: computed the network SNR = 20


In [48]:
plt.figure()
plt.xlabel('Time[sec]')
plt.ylabel('Normalised strain')
time_axis=np.linspace(0,T_obs, len(ts_noise_whiten[0])) 
plt.plot(time_axis, ts_noise_whiten[0],label='Noise',linewidth=0.01)  
plt.plot(time_axis, ts_new_whiten[0],color='darkorange',label = rf'Signal({int(par_new.m1)}$M_\odot$ + {int(par_new.m2)}$M_\odot$), SNR = {snr}',linewidth=0.5)  
plt.xlim(0,T_obs)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()
plt.savefig('signal'+'.pdf')
plt.show()

In [49]:
# ts, par,masses=sim_data(fs,T_obs)

In [50]:
#plot signal for training
# strian=np.expand_dims(ts[0], 1) 
# h=strian[0][0][0]
# plt.figure()
# plt.xlabel('Time[sec]')
# plt.ylabel('Normalised strain')
# plt.plot(time_axis,h,linewidth=0.01)  
# plt.xlim(0,T_obs)
# plt.grid(True, linestyle="--", alpha=0.5)
# plt.title("Simtulation Data")
# plt.savefig('simdata'+'.pdf')
# plt.show()

In [51]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(f'device:{device}')
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

# 优化器参数
lr = 0.001 
total_epochs = 100 

output_freq = 1  
Nnoise=25
batchsize=8
train_nsample_perepoch =10000
test_nsample_perepoch =1000

dataset_train = DatasetGenerator(fs=fs,T=T_obs,snr=snr,Nnoise=Nnoise,mdist='gh',nsample_perepoch=train_nsample_perepoch)  
dataset_test = DatasetGenerator(fs=fs,T=T_obs,snr=snr,Nnoise=Nnoise,mdist='gh',nsample_perepoch=test_nsample_perepoch)  

# 创建一个DataLoader
data_loader = DataLoader(dataset_train, batch_size=batchsize, shuffle=True,) 
test_iter = DataLoader(dataset_test, batch_size=batchsize, shuffle=True,) 

device:cuda:1
True
NVIDIA GeForce RTX 4090
GPU available? True
GPU available? True


In [52]:
#choose model for training
model_type='ResNet152' #CNN,ResNet50,ResNet101,ResNet152,LSTM

# 模型和损失历史的输出路径
checkpoint_dir = f'./checkpoints_{model_type}/'


net, epoch, train_loss_history = load_model(checkpoint_dir,model_type=model_type)  
net.to(device); 

total_epochs += epoch 
# 训练模型
train_list_l=[]
train_list_acc=[]
test_list_acc=[] 
test_list_l=[] 
train_list_acc,train_list_l,test_list_acc,test_list_l,train_masses=train(net, lr, train_nsample_perepoch, epoch, total_epochs,
      dataset_train, data_loader,test_iter,
      train_loss_history, checkpoint_dir,device, notebook=True)
print(f'total epochs={total_epochs}')

loss 0.0013, train acc 0.997, test acc 0.987
247.2 examples/sec on cuda:1
total epochs=100


In [53]:
plt.figure()
plt.plot(train_list_acc,color='darkorange',label='training data',linewidth=1.5)
plt.plot(test_list_acc,color='royalblue',label='testing data',linewidth=1.5)
plt.title(f'Total Epoch={total_epochs}')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()
plt.savefig(f'acc of {model_type}'+'.pdf')
plt.show()

In [54]:
plt.figure()
plt.plot(train_list_l,color='darkorange',label='training data',linewidth=1.5)
plt.plot(test_list_l,color='royalblue',label='testing data',linewidth=1.5)
plt.title(f'Total Epoch={total_epochs}')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()
plt.savefig(f'loss of {model_type}'+'.pdf')
plt.show()

In [55]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve

def evaluate_gpu(net, data_iter, device=None):
    
    if isinstance(net, nn.Module):
        net.eval() 
        if not device:
            device = next(iter(net.parameters())).device  
    softmax = nn.Softmax(dim=-1) 
    y_hat_list = []  
    y_list = []  
    with torch.no_grad():  
        for X, y in data_iter: 
            X = X.to(device).to(torch.float) 
            y = y.to(device).to(torch.long)  
            y_hat = net(X)  

            preds = softmax(y_hat).cpu().numpy()[:,1].tolist() 
            labels = y.cpu().numpy().tolist()

            y_hat_list.extend(preds)  
            y_list.extend(labels)  
    return np.asarray(y_hat_list), np.asarray(y_list)

In [56]:
# load model
net, epoch, train_loss_history = load_model(checkpoint_dir,model_type=model_type)
net.to(device);

/data/home/innovation/ouyangmalin/GW data analysis /baseline/BBH/test 1/main.py:293: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(p / files[0])


Load network from checkpoints_ResNet152/model_e73.pt


In [57]:
from itertools import cycle
colors = cycle(["deeppink", "aqua", "darkorange", "cornflowerblue"])

def sigma(n, tp):
    return np.sqrt(tp*(1.-tp)/n)
    
snr_list = [5, 10, 15, 20]
test_masses=[]
plt.figure()
for snr in tqdm(snr_list):
    dataset_test = DatasetGenerator(snr=snr, nsample_perepoch=test_nsample_perepoch, verbose=False)
    data_iter = DataLoader(dataset_test, batch_size=32, shuffle=True)
    y_hat_list, y_list = evaluate_gpu(net, data_iter, device)

    fpr, tpr, thresholds = roc_curve(y_list, y_hat_list,drop_intermediate=
                                     False)
    fpr_error = [sigma(len(y_list)/2., t) for t in fpr]
    tpr_error = [sigma(len(y_list)/2., t) for t in tpr]

    test_masses.append(dataset_test.masses)
    
    auc = roc_auc_score(y_list, y_hat_list)  
    color = next(colors)
    plt.plot(fpr, tpr, color=color, label=f'SNR={snr} (AUC={auc:.2f})')
    plt.fill_between(fpr, tpr+tpr_error, tpr-tpr_error, alpha=0.2, facecolor=color, zorder=0)
plt.plot(*(np.linspace(0,1,100),)*2, 'k--', label='Luck (AUC=0.50)')
# plt.axis("square")
plt.xscale('log')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curves")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(frameon=False)
plt.savefig(f'ROC1 of {model_type}'+'.pdf')
plt.show()

/tmp/ipykernel_4159714/1325050731.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()
100%|██████████| 4/4 [00:36<00:00,  9.16s/it]


In [58]:
snr_list = [2, 4, 6]
plt.figure()
for snr in tqdm(snr_list):
    dataset_test = DatasetGenerator(snr=snr, nsample_perepoch=test_nsample_perepoch, verbose=False)
    data_iter = DataLoader(dataset_test, batch_size=32, shuffle=True)
    y_hat_list, y_list = evaluate_gpu(net, data_iter, device)

    fpr, tpr, _ = roc_curve(y_list, y_hat_list,drop_intermediate=False)
    auc = roc_auc_score(y_list, y_hat_list)
    
    test_masses.append(dataset_test.masses)
    
    fpr_error = [sigma(len(y_list)/2., t) for t in fpr]
    tpr_error = [sigma(len(y_list)/2., t) for t in tpr]
    
    color = next(colors)
    plt.plot(fpr, tpr, color=color, label=f'SNR={snr} (AUC={auc:.2f})')
    plt.fill_between(fpr, tpr+tpr_error, tpr-tpr_error, alpha=0.2, facecolor=color, zorder=0)
plt.plot(*(np.linspace(0,1,100),)*2, 'k--', label='Luck (AUC=0.50)')
# plt.axis("square")
plt.xscale('log')
plt.yscale('log')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curves")
plt.grid(True, linestyle="--", alpha=0.5)
plt.xlim(1e-4,1)
plt.ylim(1e-4,1)
plt.legend(frameon=False)
plt.savefig(f'ROC2 of {model_type}'+'.pdf')
plt.show()

100%|██████████| 3/3 [00:24<00:00,  8.05s/it]


In [59]:
snr_list = np.arange(1, 21, 1)  
fap_levels = [0.2, 0.1, 0.05]  

tap_bbh = {fap: [] for fap in fap_levels}

plt.figure()
for snr in tqdm(snr_list):
    dataset_test = DatasetGenerator(fs=fs, T=T_obs, snr=snr, mdist='gh', Nnoise=Nnoise, nsample_perepoch=test_nsample_perepoch)
    data_iter = DataLoader(dataset_test, batch_size=batchsize, shuffle=True)

    y_hat_list, y_list = evaluate_gpu(net, data_iter, device)
    fpr, tpr, _ = roc_curve(y_list, y_hat_list,drop_intermediate=False)

    for fap in fap_levels:
        idx = np.argmin(np.abs(fpr - fap)) 
        tap_bbh[fap].append(tpr[idx])  

for fap in fap_levels:
    color = next(colors)
    plt.plot(snr_list, tap_bbh[fap], label=f"BBH FAP={fap}", linestyle="-", color=color,marker='o')

plt.xlabel("Optimal Matched-Filter SNR", fontsize=12)
plt.ylabel("True Alarm Probability", fontsize=12)
plt.title("Sensitivity Curves for GW Detection", fontsize=14)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, linestyle="--", alpha=0.5)
plt.xlim(1,20)
plt.ylim(1e-4,1)
plt.legend(frameon=False)
plt.savefig(f'Sensitivity Curves of {model_type}'+'.pdf')
plt.show()

  0%|          | 0/20 [00:00<?, ?it/s]

GPU available? True


  5%|▌         | 1/20 [00:06<02:12,  6.99s/it]

GPU available? True


 10%|█         | 2/20 [00:14<02:15,  7.50s/it]

...

 90%|█████████ | 18/20 [02:08<00:13,  6.98s/it]

GPU available? True


 95%|█████████▌| 19/20 [02:17<00:07,  7.30s/it]

GPU available? True


100%|██████████| 20/20 [02:24<00:00,  7.21s/it]


In [60]:
#plot mass distribution
combined_list1=[]
combined_list2=[]
for i in range(len(test_masses)):
    combined_list1.extend(test_masses[i])
for i in range(len(train_masses)):
    combined_list2.extend(train_masses[i])
train_m1,train_m2=zip(*(combined_list2))
test_m1, test_m2=zip(*(combined_list1+dataset_test.masses))
plt.figure()
plt.scatter(train_m1, train_m2, s=5, alpha=0.8,color='darkorange',label="Training Mass distribution")
plt.scatter(test_m1, test_m2, s=5, alpha=0.8,color='royalblue',label="Test Mass distribution")
plt.plot([0, max(train_m1)], [0, max(train_m1)], 'k--', label=r"$M_1 =M_2$")  
plt.xlabel(r"$M_1$ [$M_\odot$]")
plt.ylabel(r"$M_2$ [$M_\odot$]")
plt.ylim(5,100)
plt.xlim(5,100)
plt.legend()
plt.title("Mass Distribution of BBH")
plt.grid(True, linestyle="--", alpha=0.5)
plt.savefig(f'masses of {model_type}'+'.pdf')           
plt.show()
#data size
print(f'Traning Data Size={len(train_m1)}')
print(f'Testing Data Size={len(test_m1)}')

Traning Data Size=20000
Testing Data Size=160
